# Create word embeddings from small error messages

In [1]:
import utils as ut

In [2]:
reload(ut)

<module 'utils' from 'utils.pyc'>

## 1. Load the data from HDFS with the WMArchive entries

In [2]:
#timerange = [20180706, 20180906]
#timerange = [20171011, 20190401]
#timerange = [20190207, 20190307]
timerange = [20180704, 20181004]
#timerange = [20170101, 20171009]
dirs = ut.getDirs( timerange )

In [104]:
print dirs

['hdfs:///cms/wmarchive/avro/fwjr/2018/07/04', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/05', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/06', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/07', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/08', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/09', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/10', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/11', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/12', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/13', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/14', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/15', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/16', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/17', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/18', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/19', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/20', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/21', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/22', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/23', 'hdfs:///cms/wmarchive/avro/fwjr/2018/07/24', 'hdfs:///cms/wmarchive/avro/fwjr/

In [3]:
schema_file = 'hdfs:///cms/wmarchive/avro/schema.avsc'
#schema_file = 'hdfs:///cms/wmarchive/avro/schemas/current.avsc.20161215'
rdd = sc.textFile(schema_file, 1).collect()

In [106]:
# define input avro schema, the rdd is a list of lines (sc.textFile similar to readlines)
avsc = reduce(lambda x, y: x + y, rdd) # merge all entries from rdd list
schema = ''.join(avsc.split()) # remove spaces in avsc map
conf = {"avro.schema.input.key": schema}

In [107]:
# define newAPIHadoopFile parameters, java classes
aformat="org.apache.avro.mapreduce.AvroKeyInputFormat"
akey="org.apache.avro.mapred.AvroKey"
awrite="org.apache.hadoop.io.NullWritable"
aconv="org.apache.spark.examples.pythonconverters.AvroWrapperToJavaConverter"

In [108]:
data_path = dirs
# load data from HDFS
if  isinstance(data_path, list):
    avro_rdd = sc.union([sc.newAPIHadoopFile(f, aformat, akey, awrite, aconv, conf=conf) for f in data_path])
else:
    avro_rdd = sc.newAPIHadoopFile(data_path, aformat, akey, awrite, aconv, conf=conf)

## 2. Filter out the failing tasks and create a DF

In [99]:
# filter the tasks - keep only failing 
def getFailing(row):
    rec = row[0]
    meta = rec.get('meta_data', {})
    if meta.get('jobstate', '') != 'jobfailed':
        return False
    return True

# create task, site, error key  
def avro_rdd_KV(row):
    rec = row[0]
    task = rec["task"]
    steps = rec.get('steps', [])
    status = []
    sites = []
    error_msg = []
    
    for step in steps:
        errors = step['errors']
        #details = []
        #exitCodes = []
        for error in errors:
            #details.append(error['details'])
            #exitCodes.append(error['exitCode'])
            exitCode = error['exitCode']
            msg = error['details'].replace("\n", " ").replace('\r', ' ')
            sites.append( step.get('site','') )
            status.append( exitCode )
            error_msg.append( msg )
        #details = list(set(details))
        #flat_msg = ''
        #for d in details:
        #    flat_msg += d
        #flat_msg = flat_msg.replace("\n", " ").replace('\r', ' ')
        #error_msg.append(flat_msg)
        #exitCodes = list(set(exitCodes))
        
        """
        if len(exitCodes) < 1:
            exitCode = -1
        else:
            exitCode = exitCodes[0]
        """
        """
        exitCode = -9999
        if not len(exitCodes) > 1:
            exitCode = exitCodes[0]
        status.append( exitCode )
        """
        #status.append( exitCodes )
        #status.append( step.get('exitCode','') )
        #sites.append( step.get('site','') )
    #return (task, rec)
    
    return [(task, site, error, msg) for error, msg, site in zip(status, error_msg, sites) ]

In [70]:
# Filter the data
failing_workflows = avro_rdd.filter(lambda x : getFailing(x)).flatMap(lambda x : avro_rdd_KV(x))

In [23]:
failing_workflows_df_test = failing_workflows.toDF(["task_name", "site", "error", "error_msg"])

In [25]:
failing_workflows_df_test.show()

+--------------------+----+-----+--------------------+
|           task_name|site|error|           error_msg|
+--------------------+----+-----+--------------------+
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv_task_BT...|null|99303|Could not find jo...|
|/pdmvserv

### Debugging...

In [109]:
def filter_for_problem(row):
    #task_name = row[0]
    rec = row[0]
    task_name = rec["task"]
    #task_name = row[0]
    #error = row[2]
    #test_task = '/pdmvserv_task_HIG-RunIIFall17wmLHEGS-03477__v1_T_190208_210318_4829/HIG-RunIIFall17wmLHEGS-03477_0'
    #test_error = 99109
    #test_task = '/pdmvserv_task_B2G-RunIISummer15wmLHEGS-01375__v1_T_180703_203928_6625/B2G-RunIISummer15wmLHEGS-01375_0'
    #test_task = '/pdmvserv_task_HIG-RunIIFall17wmLHEGS-00630__v1_T_180216_120647_5907/HIG-RunIIFall17wmLHEGS-00630_0/HIG-RunIIFall17DRPremix-00737_0'
    #test_task = '/vlimant_ACDC0_task_HIG-RunIIFall17wmLHEGS-01415__v1_T_180706_002124_986/HIG-RunIIFall17DRPremix-02001_1/HIG-RunIIFall17DRPremix-02001_1MergeAODSIMoutput/HIG-RunIIFall17MiniAODv2-01299_0'
    #test_error = 85
    #test_task = '/pdmvserv_task_HIG-RunIIFall17wmLHEGS-00689__v1_T_180307_201555_2433/HIG-RunIIFall17wmLHEGS-00689_0/HIG-RunIIFall17DRPremix-00886_0'
    test_task = '/pdmvserv_task_HIG-RunIIFall17wmLHEGS-02145__v1_T_180705_162228_8813/HIG-RunIIFall17wmLHEGS-02145_0/HIG-RunIIFall17DRPremix-02708_0'
    if (task_name == test_task): # and (error == test_error):
        return True
    else:
        return False

In [110]:
failures = avro_rdd.filter(lambda x : getFailing(x)).filter(lambda x : filter_for_problem(x))
#failures = avro_rdd.filter(lambda x : getFailing(x)).flatMap(lambda x : avro_rdd_KV(x)).filter(lambda x : filter_for_problem(x))

In [111]:
failures.saveAsTextFile("hdfs:///cms/users/llayer/debug5.csv")

In [43]:
probs = failures.collect()

In [44]:
print len(probs)

12


In [45]:
for prob in probs:
    print prob[1], prob[2]

T1_UK_RAL 85
T1_UK_RAL 85
T1_UK_RAL 85
T1_UK_RAL 85
T1_UK_RAL 85
T1_UK_RAL 85
T1_FR_CCIN2P3 85
T1_FR_CCIN2P3 85
T1_UK_RAL 85
T1_UK_RAL 85
T1_UK_RAL 85
T1_UK_RAL 85


### Reduce

In [71]:
def map_to_key(row):
    return (row[0], row[1], row[2]), row[3]

In [45]:
print failing_workflows.map(lambda x : map_to_key(x)).take(1)

[((u'/pdmvserv_task_SMP-RunIISummer16DR80Premix-00011__v1_T_161227_083845_4347/SMP-RunIISummer16DR80Premix-00011_0', u'T2_UK_London_IC', 99999), u'Could not find report file for step stageOut1!')]


In [72]:
def reduceToLongest(row):
    key = row[0]
    message_list = list(row[1])
    longest_msg = ''
    for msg in message_list:
        if len(msg) > len(longest_msg):
            longest_msg = msg
    return key[0], key[1], key[2], longest_msg

failing_workflows_reduce = failing_workflows.map(lambda x : map_to_key(x)).groupByKey().map(lambda x : reduceToLongest(x))

In [73]:
failing_workflows_df = failing_workflows_reduce.toDF(["task_name", "site", "error", "error_msg"])

In [74]:
failing_workflows_df.show()

+--------------------+---------------+-----+--------------------+
|           task_name|           site|error|           error_msg|
+--------------------+---------------+-----+--------------------+
|/pdmvserv_task_BP...|     T2_DE_DESY|   80|  Adding last 25 ...|
|/fabozzi_Run2016H...|   T2_US_Purdue|99109|Error in StageOut...|
|/pdmvserv_task_B2...|           null|99303|Could not find jo...|
|/pdmvserv_task_HI...|T2_FR_GRIF_IRFU|  134|  Adding last 25 ...|
|/fabozzi_Run2016B...|     T1_RU_JINR|99999|Adding extra erro...|
|/pdmvserv_task_SU...|      T1_UK_RAL|  139|  Adding last 25 ...|
|/pdmvserv_task_EX...|T2_US_Wisconsin| 8001|Exit 8001: CMSExe...|
|/prebello_Run2016...|   T2_US_Purdue| 8022|An exception of c...|
|/prozober_ACDC0_t...|     T2_CH_CERN|99999|Adding extra erro...|
|/pdmvserv_task_HI...|     T2_DE_RWTH| 8028|An exception of c...|
|/pdmvserv_task_EG...|     T2_US_UCSD|   85|  Adding last 25 ...|
|/pdmvserv_task_B2...|      T1_UK_RAL| 8001|Exit 8001: CMSExe...|
|/pdmvserv

In [63]:
failing_workflows_df.rdd.getNumPartitions()

4159

In [66]:
failing_workflows_df_rep = failing_workflows_df.repartition(500)
print failing_workflows_df_rep.rdd.getNumPartitions()

500


## 3. Load actionhist and convert to df

In [75]:
from pyspark import SQLContext, StorageLevel
sql = SQLContext(sc)
labeled_failing_tasks = (sql.read
     .format("com.databricks.spark.csv")
     .option("header", "true")
     .load("hdfs:///cms/users/llayer/actionhist.csv"))

#rdd_failing_tasks = rdd_failing_tasks.rdd.map(tuple)
#print rdd_failing_tasks.take(1)

In [76]:
from pyspark.sql.types import IntegerType
labeled_failing_tasks = labeled_failing_tasks.withColumn("error", labeled_failing_tasks["error"].cast(IntegerType()))
print labeled_failing_tasks.head()

Row(task_name=u'/vlimant_ACDC0_task_HIG-RunIIFall17wmLHEGS-01415__v1_T_180706_002124_986/HIG-RunIIFall17DRPremix-02001_1/HIG-RunIIFall17DRPremix-02001_1MergeAODSIMoutput/HIG-RunIIFall17MiniAODv2-01299_0', side_state=u'good_site', error=85, site=u'T1_UK_RAL', action=u'acdc', memory=None)


## 4. Join both frames and save to HDFS

In [77]:
df = failing_workflows_df.join(labeled_failing_tasks, ['task_name','site', 'error'])

In [78]:
df = df.dropDuplicates()

In [79]:
df.write.format('com.databricks.spark.csv').save('hdfs:///cms/users/llayer/df_reduced_codes2.csv',header = 'true')